In [7]:
import os
import numpy as np
from tqdm import tqdm
import joblib
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import xgboost as xgb
import cv2

In [8]:
# Path to the dataset
DATASET_PATH = 'Silhouettes of human posture/'
# Define the classes based on folder names
CLASSES = ['bending', 'lying', 'sitting', 'standing']

In [9]:
# --- Feature Extraction Function ---
def extract_hu_moments(image_path):
    """Loads an image, preprocesses it, and extracts Hu Moments."""
    # Load image in grayscale
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if image is None:
        return None
    
    # Resize and threshold to ensure it's a binary silhouette
    image = cv2.resize(image, (100, 100))
    _, thresh = cv2.threshold(image, 127, 255, cv2.THRESH_BINARY)
    
    # Calculate moments
    moments = cv2.moments(thresh)
    
    # Calculate Hu Moments
    hu_moments = cv2.HuMoments(moments)
    
    # Log transform to make them more scale-invariant
    # Add a small epsilon to avoid log(0)
    hu_moments = -1 * np.sign(hu_moments) * np.log10(np.abs(hu_moments) + 1e-7)
    
    return hu_moments.flatten()

In [10]:
# --- Load Data and Extract Features ---
features = []
labels = []

print("Extracting features from the dataset...")
for class_idx, class_name in enumerate(CLASSES):
    class_path = os.path.join(DATASET_PATH, class_name)
    for image_name in tqdm(os.listdir(class_path), desc=f"Processing {class_name}"):
        image_path = os.path.join(class_path, image_name)
        
        hu_features = extract_hu_moments(image_path)
        
        if hu_features is not None:
            features.append(hu_features)
            labels.append(class_idx)


Extracting features from the dataset...


Processing standing: 100%|█████████████████████████████████████████████████████████| 1200/1200 [00:16<00:00, 73.66it/s]


In [12]:
# Convert to numpy arrays
X = np.array(features)
y = np.array(labels)

print(f"\nFeature extraction complete!")
print(f"Data shapes: X={X.shape}, y={y.shape}")


Feature extraction complete!
Data shapes: X=(4800, 7), y=(4800,)


In [13]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

In [14]:
# Using 'multi:softmax' for multi-class classification
print("Training the XGBoost model...")
xgb_model = xgb.XGBClassifier(
    objective='multi:softmax',
    num_class=len(CLASSES),
    eval_metric='mlogloss',
    use_label_encoder=False,
    random_state=42,
    verbose=True
)
xgb_model.fit(X_train, y_train)

print("Model training complete! 🚀")

Training the XGBoost model...


C:\Users\dhruv\anaconda3\Lib\site-packages\xgboost\training.py:183: UserWarning: [12:09:00] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder", "verbose" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Model training complete! 🚀


In [15]:
# Evaluate the model
y_pred = xgb_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"\nModel Accuracy: {accuracy * 100:.2f}%")
print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=CLASSES))


Model Accuracy: 59.69%

Classification Report:
              precision    recall  f1-score   support

     bending       0.59      0.56      0.58       240
       lying       0.60      0.53      0.56       240
     sitting       0.61      0.63      0.62       240
    standing       0.59      0.66      0.62       240

    accuracy                           0.60       960
   macro avg       0.60      0.60      0.60       960
weighted avg       0.60      0.60      0.60       960



In [16]:
# Save the model to disk
joblib.dump(xgb_model, 'posture_model.pkl')

print("Model saved as 'posture_model.pkl'")

Model saved as 'posture_model.pkl'
